In [6]:
import requests as rq
import pandas as pd

#Спробую пояснити код, щоб вам ленше було його зрозуміти

#Тут я за допомогою бібліотеки requests дістаю посилання з сайту weatherapi, який я підключив до свого ключа, бо так значно легше отримати класний json файл, ніж на рандомному сайті погоди
response = rq.get("https://api.weatherapi.com/v1/forecast.json?key=8c501b08b6db4eeeb4f101047252206&q=Kyiv&days=7&aqi=no&alerts=no")
#Перетворюю його в дані, які можна прочитати пайтону з json формату
data = response.json()

#Мені потрібно достукатися до конкретного дня (якщо поглянути на посилання, то там буде видно як це словник у словнику... у словнику), тож я створюю окрему змінну для цього
hours = data["forecast"]["forecastday"]

#Пустий список для внесення туди подальших необхідних даних
forecast_list = []
#Тепер я хочу щоб мені занесло в табличку необхідні дані. Для цього я проітерую його у два цикли, тому що перший цикл проходитиме по дням, а другий по годинам. Тобто в кожному дні є 24 години і цей цикл зробить ітерацію на кожній годині

for day in hours:
    for hour in day["hour"]:
        forecast_list.append({
            "date": hour["time"].split(" ")[0],
            "time": hour["time"].split(" ")[1],
            "temp": hour["temp_c"],
            "wind": hour["wind_kph"]
        })
#Тепер візуалізую проітерований список в датафрейм
df = pd.DataFrame(forecast_list)

#Роблю дуже цікаву штуку. За допомогою функції pivot() я можу змінити структуру мого списку, і зробити дні - індексом, а колнки температурою й швидкістю вітру, таким чином матиму відповідні дані в одній табличці
df = df.pivot(index="date", columns="time", values=["temp", "wind"])
print(df)

#Перше та друге завдання виконано! Тепер я дізнаюся найбільше, найменше й середнє значення температури за 3 дні. Я створюю відслайсований список, а далі просто за колонкою визначаю ці значення
filtere_df = df.loc["2025-06-22":"2025-06-25"]
max_temp = filtere_df["temp"].max()
min_temp = filtere_df["temp"].min()
avg_temp = filtere_df["temp"].mean()
#Так як зараз нам би видало ці значення за кожну годину, я перетворюю їх у одне єдине число другою ітерацією. iloc дозволяє достукатися до елементів списку за позицією (а не індексом)
max_temp_1 = max_temp.iloc[1].max()
min_temp_1 = min_temp.iloc[1].min()
avg_temp_1 = avg_temp.iloc[1].mean()

print(f"Average tempreture in next 3 days is {avg_temp_1}, max temperature in 3 days is {max_temp_1}, min temperature is {min_temp_1}")
#Зараз я знову створю середнє значення швидкості вітру за 7 днів
avg_wind_speed = df["wind"].mean()
avg_wind_speed_1 = avg_wind_speed.iloc[1].mean()

print(f"Average wind speed per hour in km in next 7 days is {avg_wind_speed_1}")

#Знйду значення, які будуть більшими ніж середнє
higher_than_avg_wind_speed = df[df["wind"] > avg_wind_speed_1]
#А тепер із них всіх порахую суму цих унікалиних значень (ті які не унікальні в таблиці позначалися як NaN)
number_of_hours = higher_than_avg_wind_speed["wind"].nunique().sum()

#Запринтую і готово!
print(f"The number of hours that wind speed was more than average in next 7 days is {number_of_hours}")

            temp                                                        ...  \
time       00:00 01:00 02:00 03:00 04:00 05:00 06:00 07:00 08:00 09:00  ...   
date                                                                    ...   
2025-06-22  14.1  12.6  12.0  11.6  11.4  11.9  13.2  15.2  15.1  15.7  ...   
2025-06-23  12.8  12.3  11.7  11.3  10.9  11.0  11.9  13.4  15.0  16.5  ...   
2025-06-24  14.8  14.5  14.2  14.3  14.2  14.6  15.3  16.3  18.9  20.1  ...   
2025-06-25  14.8  14.3  14.2  14.0  13.7  14.9  15.6  17.9  18.8  19.6  ...   
2025-06-26  18.0  16.9  16.2  15.6  14.9  14.8  15.5  16.9  18.3  19.6  ...   
2025-06-27  15.9  15.5  15.1  14.9  14.6  15.0  16.6  18.5  20.4  21.8  ...   
2025-06-28  17.0  16.8  16.5  15.9  15.4  14.8  15.5  16.1  16.8  17.0  ...   

            wind                                                        
time       14:00 15:00 16:00 17:00 18:00 19:00 20:00 21:00 22:00 23:00  
date                                                           

In [7]:
#Відобразимо в таблицю красиво
df

temp                                                        ...  \
time       00:00 01:00 02:00 03:00 04:00 05:00 06:00 07:00 08:00 09:00  ...   
date                                                                    ...   
2025-06-22  14.1  12.6  12.0  11.6  11.4  11.9  13.2  15.2  15.1  15.7  ...   
2025-06-23  12.8  12.3  11.7  11.3  10.9  11.0  11.9  13.4  15.0  16.5  ...   
2025-06-24  14.8  14.5  14.2  14.3  14.2  14.6  15.3  16.3  18.9  20.1  ...   
2025-06-25  14.8  14.3  14.2  14.0  13.7  14.9  15.6  17.9  18.8  19.6  ...   
2025-06-26  18.0  16.9  16.2  15.6  14.9  14.8  15.5  16.9  18.3  19.6  ...   
2025-06-27  15.9  15.5  15.1  14.9  14.6  15.0  16.6  18.5  20.4  21.8  ...   
2025-06-28  17.0  16.8  16.5  15.9  15.4  14.8  15.5  16.1  16.8  17.0  ...   

            wind                                                        
time       14:00 15:00 16:00 17:00 18:00 19:00 20:00 21:00 22:00 23:00  
date                                                                    
2025-06-22  25.2  22.3  18.7  18.4  14.8  12.6  13.3  16.2  14.4  15.1  
2025-06-23  16.2  15.5  14.4  12.6   9.7   6.5   5.0   2.2   2.2   4.3  
2025-06-24  22.3  23.4  25.2  23.8  20.9  18.0  15.8  16.9  16.2  13.7  
2025-06-25  24.8  22.0  25.9  22.3  20.2  18.7  15.1  15.5  20.2  20.5  
2025-06-26  19.8  19.4  18.7  19.8  19.1  15.5  12.6  10.1   7.6   4.7  
2025-06-27  16.6  17.3  18.4  19.1  20.2  21.6  22.7  20.2  17.3  14.8  
2025-06-28  20.5  19.4  18.4  17.3  16.9  16.2  15.8  17.3  18.7  20.2  

[7 rows x 48 columns]